In [2]:
from pathlib import *
import sqlalchemy as sa
import pandas as pd

database_management_sys = sa.create_engine("sqlite:///../database_code/data/se/main.db")
connect_dbms_to_db = database_management_sys.connect()

In [26]:
trial_tournament_set = """
    SELECT * FROM folk_theorem_experiment
    WHERE tournament_player_set = '20'
    AND noise = '0.0'
    AND player_strategy_name = 'Defector'
"""
trial_data = pd.read_sql(trial_tournament_set, connect_dbms_to_db)

In [27]:
places_of_jump_up = trial_data.index[trial_data["least_prob_of_defection"] > 0]
jump_up_ending_probs = []
for index in places_of_jump_up:
    if trial_data.iloc[index - 1]["least_prob_of_defection"] == 0:
        jump_up_ending_probs.append(trial_data.iloc[index]["prob_of_game_ending"])
jump_up_ending_probs

[]

In [44]:
len(jump_up_ending_probs)

0

In [41]:
min_threshold = max_threshold = median_threshold = mean_threshold = None

In [42]:
min_threshold, max_threshold, mean_threshold, median_threshold

(None, None, None, None)

In [28]:
player_set_collection = """
    SELECT * FROM folk_theorem_experiment
    WHERE tournament_player_set = ?
    AND noise = ?
    AND player_strategy_name = 'Defector'
"""

In [38]:
table_headings = pd.read_sql(
    "folk_theorem_experiment", connect_dbms_to_db
).columns.tolist()#
table_headings

['experiment_number',
 'number_of_players',
 'tournament_player_set',
 'player_strategy_name',
 'is_long_run_time',
 'is_stochastic',
 'memory_depth_of_strategy',
 'prob_of_game_ending',
 'payoff_matrix',
 'num_of_repetitions',
 'num_of_equilibria',
 'nash_equilibria',
 'least_prob_of_defection',
 'greatest_prob_of_defection',
 'noise',
 'warning_message']

In [36]:
collect_relevant_data = connect_dbms_to_db.execute(player_set_collection, (0, 0.0))
each_set_data = pd.DataFrame(
        collect_relevant_data.fetchall(), columns=table_headings
        )

In [39]:
each_set_data["number_of_players"][0]

2

In [ ]:
def find_min_p_thresh(tournament_set):
    """
    A function to find the minimum p-threshold from the database, where:
    
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level.
    """
    defection_prob_at_smallest_prob_game_ending = tournament_set["least_prob_of_defection"][0]
    any_equal_prob = tournament_set[tournament_set["least_prob_of_defection"] == defection_prob_at_smallest_prob_game_ending]
    if len(tournament_set["least_prob_of_defection"]) == len(any_equal_prob):
        if defection_prob_at_smallest_prob_game_ending == 0:
        # that is, if the least probability of defection is constant at 0 for all game ending probabilities.    
            min_p_thresh = None
        else:
        # the least probability of defection is equal to some non-zero constant for all game ending probabilities.
            min_p_thresh = min(tournament_set["prob_of_game_ending"])
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        min_p_thresh = min(prob_of_defection_is_non_zero["prob_of_game_ending"])
    return(min_p_thresh)

In [ ]:
def find_max_p_thresh(tournament_set):
    """
    A function to find the maximum p-threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level.
    """
    defection_prob_at_smallest_prob_game_ending = tournament_set["least_prob_of_defection"][0]
    any_equal_prob = tournament_set[tournament_set["least_prob_of_defection"] == defection_prob_at_smallest_prob_game_ending]
    if len(tournament_set["least_prob_of_defection"]) == len(any_equal_prob):
        if defection_prob_at_smallest_prob_game_ending == 0:
        # that is, if the least probability of defection is constant at 0 for all game ending probabilities.    
            max_p_thresh = None
        else:
        # the least probability of defection is equal to some non-zero constant for all game ending probabilities.
            max_p_thresh = max(tournament_set["prob_of_game_ending"])
    else:
        prob_of_defection_is_zero = tournament_set[tournament_set["least_prob_of_defection"] == 0]
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        max_zero_defection_game_ending = max(prob_of_defection_is_zero["prob_of_game_ending"])
        if len(prob_of_defection_is_non_zero[prob_of_defection_is_non_zero["prob_of_game_ending"] > max_zero_defection_game_ending]["prob_of_game_ending"]) == 0:
            max_p_thresh = max(prob_of_defection_is_non_zero["prob_of_game_ending"])
        else:    
            max_p_thresh = min(prob_of_defection_is_non_zero[prob_of_defection_is_non_zero["prob_of_game_ending"] > max_zero_defection_game_ending]["prob_of_game_ending"])
    return(max_p_thresh)

In [ ]:
def find_mean_p_thresh(tournament_set, min_p_thresh, max_p_thresh):
    """
    A function to find the mean p-threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level;
        "min_p_thresh" is a numeric variable between 0 and 1 indicating the minimum p-threshold; and
        "max_p_thresh" is a numeric variable between 0 and 1 indicating the maximum p-threshold.
    """
    if min_p_thresh == max_p_thresh:
        mean_p_thresh = min_p_thresh
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        non_zero_defection_prob_between_min_max = prob_of_defection_is_non_zero[(prob_of_defection_is_non_zero["prob_of_game_ending"] >= min_p_thresh) & (prob_of_defection_is_non_zero["prob_of_game_ending"] <= max_p_thresh)]
        mean_p_thresh = non_zero_defection_prob_between_min_max["prob_of_game_ending"].mean()
    return(mean_p_thresh)

In [ ]:
def find_median_p_thresh(tournament_set, min_p_thresh, max_p_thresh):
    """
    A function to find the median p_threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level;
        "min_p_thresh" is a numeric variable between 0 and 1 indicating the minimum p-threshold; and
        "max_p_thresh" is a numeric variable between 0 and 1 indicating the maximum p-threshold.
    """
    if min_p_thresh == max_p_thresh:
        median_p_thresh = min_p_thresh
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        non_zero_defection_prob_between_min_max = prob_of_defection_is_non_zero[(prob_of_defection_is_non_zero["prob_of_game_ending"] >= min_p_thresh) & (prob_of_defection_is_non_zero["prob_of_game_ending"] <= max_p_thresh)]
        median_p_thresh = non_zero_defection_prob_between_min_max["prob_of_game_ending"].median()
    return(median_p_thresh)

In [ ]:
min_p_thresh = find_min_p_thresh(tournament_set=trial_data)
max_p_thresh = find_max_p_thresh(tournament_set=trial_data)
mean_p_thresh = find_mean_p_thresh(tournament_set=trial_data, min_p_thresh=min_p_thresh, max_p_thresh=max_p_thresh)
median_p_thresh = find_median_p_thresh(tournament_set=trial_data, min_p_thresh=min_p_thresh, max_p_thresh=max_p_thresh)

min_p_thresh, max_p_thresh, mean_p_thresh, median_p_thresh